<a href="https://colab.research.google.com/github/karsarobert/ChatGPT2025/blob/main/11/ChatGPT_11_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chat GPT és más nagy nyelvi modellek alkalmazása
##PTE Gépi tanulás
###11. gyakorlat: Langchain 2. rész
2025. április 28.

# Langchain oktatóanyag: LLM alkalmazások fejlesztése

Ez az anyag a Langchain keretrendszer használatát mutatja be, amely lehetővé teszi nagy nyelvi modelleken (LLM-eken) alapuló alkalmazások fejlesztését. Kitérünk az alapvető komponensekre, az ügynökök (agents) használatára, a memória kezelésére és a tudásbázisok integrálására.

**Fontos megjegyzés:** A Langchain könyvtár gyorsan fejlődik. Az eredeti jegyzetfüzetben [cite: 1] található szintaxis és néhány osztály elavult lehet. Ez a frissített változat a modernebb, Langchain Expression Language (LCEL) alapú megközelítéseket mutatja be.

---

## 1. Telepítés és beállítás

Az első lépés a szükséges Python csomagok telepítése.

In [1]:
# Szükséges csomagok telepítése
!pip install openai langchain google-search-results pypdf tiktoken faiss-cpu langchain-community langchain-openai langchain-huggingface langchainhub -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━

**Magyarázat:**
* `openai`, `langchain-openai`: Az OpenAI API és modellek használatához.
* `langchain`: A fő keretrendszer.
* `google-search-results`: A SerpAPI használatához (Google keresés).
* `pypdf`: PDF dokumentumok betöltéséhez.
* `tiktoken`: Tokenizáláshoz (szöveg darabolása, amit az LLM megért).
* `faiss-cpu`: Vektoradatbázis létrehozásához és kezeléséhez (CPU verzió).
* `langchain-community`: Közösségi integrációk (pl. Wrapperek, Loaderek).
* `langchain-huggingface`: Hugging Face modellek integrálásához.
* `langchainhub`: Előre definiált prompt sablonok letöltéséhez.

Ezután be kell állítani az API kulcsokat (pl. OpenAI, SerpAPI, Hugging Face). Google Colabban ezt biztonságosan megteheted a bal oldali "Secrets" (kulcs) ikon alatt, majd a kódban a `google.colab.userdata` segítségével érheted el őket.

In [2]:
# API Kulcsok beolvasása (Colab Secrets használata ajánlott)
from google.colab import userdata
import os

# Helyettesítsd be a Secret neveket, amiket Colabban megadtál
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

# VAGY közvetlenül változókba (kevésbé biztonságos)
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # Cseréld le a Colab Secret nevére
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY') # Cseréld le a Colab Secret nevére
HF_TOKEN = userdata.get('HF_TOKEN')             # Cseréld le a Colab Secret nevére

---

## 2. ReAct ügynökök (Agents)

A ReAct (Reason + Act) egy paradigma, ahol az LLM nemcsak választ generál, hanem "gondolkodik" a lépésekről és külső eszközöket (tools) hív meg információgyűjtés vagy műveletvégrehajtás céljából.

**Példa:**

In [3]:
# Szükséges importok
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper # Javított import
from langchain.agents import AgentExecutor, create_react_agent, Tool # Tool importálása
from langchain import hub # A prompt sablonokhoz
from google.colab import userdata # Vagy os az api kulcsokhoz

# 1. LLM inicializálása
# Győződj meg róla, hogy az OPENAI_API_KEY be van állítva (pl. userdata-ból)
llm = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model="gpt-3.5-turbo")

# 2. Eszköz(ök) definiálása
# Győződj meg róla, hogy a SERPAPI_API_KEY be van állítva
search = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))
tools = [
    Tool(
        name="Search", # Az eszköz neve az ügynök számára
        func=search.run, # A meghívandó függvény
        description="hasznos, ha aktuális eseményekkel vagy naprakész információkkal kapcsolatos kérdésekre kell válaszolni" # Fontos a jó leírás!
    )
]

# 3. Prompt Sablon betöltése (ReAct-hoz)
# A hub.pull letölt egy előre definiált prompt sablont a Langchain Hub-ról
# Ez a sablon tartalmazza az instrukciókat az LLM számára, hogyan gondolkodjon (Thought) és cselekedjen (Action).
prompt = hub.pull("hwchase17/react")

# 4. Ügynök létrehozása
# Ez a függvény összekapcsolja az LLM-et, az eszközöket és a promptot
agent = create_react_agent(llm, tools, prompt)

# 5. Agent Executor létrehozása
# Ez futtatja az ügynököt ciklikusan:
# - Az LLM generál egy gondolatot (Thought) és egy műveletet (Action).
# - Az Executor végrehajtja a műveletet (meghívja a Tool-t).
# - Az eredmény (Observation) visszakerül az LLM-hez a következő ciklushoz.
# - Ez addig folytatódik, amíg az LLM egy végső választ (Final Answer) nem ad.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose=True kiírja a gondolkodási folyamatot

# 6. Futtatás
# Az invoke metódus bemenete egy szótár, ahol a kulcs(ok)nak meg kell egyeznie a prompt által vártakkal (itt 'input')
response = agent_executor.invoke({"input": "Ki volt Arany János felesége és mikor született?"})
print("\nVégső válasz:")
print(response['output'])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Először keresnem kell Arany János feleségének nevét, majd utána az ő születési dátumát.
Action: Search
Action Input: "Arany János felesége"Julianna ErcseyMost már tudom, hogy Arany János felesége Julianna Ercsey volt.
Action: Search
Action Input: "Julianna Ercsey születési dátuma"May 1, 1818Most már tudom, hogy Arany János felesége, Julianna Ercsey 1818. május 1-jén született.
Final Answer: Arany János felesége Julianna Ercsey volt, 1818. május 1-jén született.

> Finished chain.

Végső válasz:
Arany János felesége Julianna Ercsey volt, 1818. május 1-jén született.


**Magyarázat:**
* Az ügynök az `Tool` `description`-je alapján dönti el, mikor melyik eszközt használja. Ezért kulcsfontosságú a pontos és informatív leírás.
* A `create_react_agent` és `AgentExecutor` a modern LCEL alapú megközelítés ügynökök létrehozására.
* A `verbose=True` kapcsolóval követhető az ügynök "gondolkodási folyamata": `Thought` (Gondolat), `Action` (Művelet - melyik eszközt hívja), `Action Input` (Művelet bemenete), `Observation` (Megfigyelés - az eszköz válasza). Ez rendkívül hasznos hibakereséshez.

---

## 3. További ügynöktípusok (áttekintés)

A Langchain többféle ügynök logikát támogat:

* **Structured Input ReAct:** Strukturáltabb bemenetet váró eszközökhöz.
* **OpenAI Functions/Tools:** Kifejezetten OpenAI modellekhez tervezett ügynökök, amelyek kihasználják a modellek beépített "function calling" vagy "tool calling" képességét a megbízhatóbb eszközhasználat érdekében. Ez gyakran a legajánlottabb módszer OpenAI modellekkel.
* **Conversational:** Memóriával rendelkező ügynökök, amelyek emlékeznek a beszélgetés korábbi részeire. Lásd később a Memória résznél.
* **Self-ask with search:** Összetett kérdéseket bont al-kérdésekre, amelyeket kereséssel válaszol meg.
* **ReAct document store:** Dokumentum-alapú kérdésválaszolásra optimalizált ReAct ügynök (manapság ezt gyakran RAG láncokkal valósítják meg).
* **Plan-and-execute:** Először tervet készít, majd lépésenként végrehajtja azt (komplexebb feladatokra).

---

## 4. Hugging Face modellek használata ügynökként

A Langchain nemcsak OpenAI modellekkel működik. Integrálható nyílt forráskódú modellekkel is a Hugging Face Hub-ról.

**Példa:**

In [4]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain import hub
from google.colab import userdata

# 1. Hugging Face Endpoint LLM inicializálása
# Figyelem: Hozzáférést kell kérni a modellhez a Hugging Face-en!
# Győződj meg róla, hogy a HF_TOKEN be van állítva
llm_hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", # Példa modell, cserélhető
    task="text-generation", # A feladat típusa
    huggingfacehub_api_token=userdata.get("HF_TOKEN"),
    max_new_tokens=256, # Növelhetjük a generált tokenek számát
    temperature=0.7 # Hőmérséklet beállítása (kreativitás vs pontosság)
)

# 2. Eszközök (ugyanaz, mint korábban)
# Győződj meg róla, hogy a SERPAPI_API_KEY be van állítva
search_hf = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))
tools_hf = [
    Tool(
        name="Search",
        func=search_hf.run,
        description="hasznos, ha aktuális eseményekkel vagy naprakész információkkal kapcsolatos kérdésekre kell válaszolni"
    )
]

# 3. Prompt (ugyanaz, mint korábban)
prompt_hf = hub.pull("hwchase17/react")

# 4. Ügynök és Executor létrehozása
agent_hf = create_react_agent(llm_hf, tools_hf, prompt_hf)
agent_executor_hf = AgentExecutor(
    agent=agent_hf,
    tools=tools_hf,
    verbose=True,
    handle_parsing_errors=True # Ajánlott hibakezeléshez
)

# 5. Futtatás
# Előfordulhat, hogy a nyílt modelleknek több próbálkozásra van szükségük
# vagy finomhangolni kell a promptot a megbízható működéshez.
try:
    response_hf = agent_executor_hf.invoke({"input": "How many people live in Canada as of 2023?"})
    print("\nVégső válasz (Hugging Face):")
    print(response_hf['output'])
except Exception as e:
    print(f"\nHiba történt a Hugging Face ügynök futtatása közben: {e}")
    print("Lehetséges okok: API hiba, modell nem elérhető, prompt/parse hiba.")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/inference/_client.py:2288: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(




> Entering new AgentExecutor chain...
 I don't have real-time data, so I need to use the Search tool to find the most recent population data for Canada.
Action: Search
Action Input: Canada population 2023
Observation40.1 million

/usr/local/lib/python3.11/dist-packages/huggingface_hub/inference/_client.py:2288: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


40.1 million is the estimated population of Canada in 2023.
Final Answer: As of 2023, approximately 40.1 million people live in Canada.</s>

> Finished chain.

Végső válasz (Hugging Face):
As of 2023, approximately 40.1 million people live in Canada.</s>


**Magyarázat és hibaelhárítás:**
* A `HuggingFaceEndpoint` lehetővé teszi a Hugging Face által hosztolt modellek használatát API-n keresztül. Szükséges hozzá egy Hugging Face API token és jogosultság a választott modellhez.

* A `handle_parsing_errors=True` az `AgentExecutor`-ban segíthet kezelni azokat az eseteket, amikor a modell kimenete nem felel meg a várt formátumnak, és egy alapértelmezett hibaüzenetet ad vissza az ügynöknek.

---

## 5. Beszélgetési láncok és memória

Ahhoz, hogy egy chatbot emlékezzen a korábbi interakciókra, memóriát kell hozzáadni. Az LCEL segítségével rugalmasan építhetünk memóriával rendelkező láncokat.

**Példa (LCEL alapú):**

In [10]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage
from google.colab import userdata

# 1. LLM
# Győződj meg róla, hogy az OPENAI_API_KEY be van állítva
llm_chat = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model="gpt-3.5-turbo")

# 2. Memória
# A ConversationBufferMemory tárolja az összes üzenetet.
# return_messages=True fontos, hogy Message objektumokat adjon vissza, ne csak egy stringet.
memory_chat = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 3. Prompt Sablon Memóriával
# A MessagesPlaceholder jelöli a helyet, ahova a memóriából betöltött üzenetek kerülnek.
prompt_chat = ChatPromptTemplate.from_messages([
    SystemMessage(content="Ön egy barátságos és segítőkész asszisztens."), # Rendszerüzenet a chatbot személyiségének beállításához
    MessagesPlaceholder(variable_name="chat_history"), # Ide kerül a memória tartalma
    ("human", "{input}") # Az aktuális felhasználói input helye
])

# 4. LCEL Lánc létrehozása Memóriával
# A RunnablePassthrough.assign dinamikusan hozzáadja a memóriából betöltött előzményeket a lánc inputjához.
# A lambda függvény biztosítja, hogy a memória mindig az aktuális állapotot töltse be.
chain_with_memory = RunnablePassthrough.assign(
    # A 'memory_chat.load_memory_variables({})' betölti az összes tárolt üzenetet.
    # A '{}' üres szótár, mert az inputot a lánc külső része adja.
    # A kimenet egy szótár {'chat_history': [...]}, ebből vesszük az üzenetlistát.
    chat_history=lambda x: memory_chat.load_memory_variables(x).get("chat_history", [])
) | prompt_chat | llm_chat

# 5. Beszélgetés és Memória kezelése
def chat_with_memory_func(user_input):
    # Lánc meghívása az aktuális felhasználói inputtal
    # A lánc a RunnablePassthrough miatt megkapja a {"input": user_input} szótárat,
    # amihez hozzárendeli a 'chat_history'-t.
    response = chain_with_memory.invoke({"input": user_input})

    # Memória frissítése: elmentjük a felhasználó kérdését ÉS az AI válaszát
    # A save_context két szótárat vár: az inputokat és az outputokat.
    memory_chat.save_context({"input": user_input}, {"output": response.content})

    return response.content

# Használat:
print("AI:", chat_with_memory_func("Szia! Segítenél nekem?"))
print("AI:", chat_with_memory_func("Szeretnék Bécsbe utazni 2 napra. Mit javasolsz megnézni?"))
print("AI:", chat_with_memory_func("Említetted a Prátert. Mesélnél róla bővebben?")) # Itt már emlékeznie kell az előző kontextusra

#Memória tartalmának ellenőrzése (opcionális)
print("\nMemória tartalma:")
print(memory_chat.load_memory_variables({}))

AI: Szia! Természetesen, hogyan segíthetek?
AI: Bécs egy gyönyörű város, tele különleges látnivalókkal! Ha csak 2 napod van, itt van néhány javaslatom:

1. Schönbrunni Palota és kertje: Egy lenyűgöző barokk palota és gyönyörű kertje, amelyet mindenképpen érdemes megnézni.

2. Belvedere Palota: A festménygyűjteménnyel és a lenyűgöző kerttel rendelkező barokk palota, ahol számos műalkotást tekinthetsz meg.

3. Stephansdom: Bécs legismertebb székesegyháza, mely lenyűgöző építészeti részletekkel rendelkezik.

4. Hofburg Palota: Az egykori császári palota, mely ma múzeumoknak és hivataloknak ad otthont.

5. Práter: Bécs híres vidámparkja, ahol szórakozni lehet és kipróbálni egy sor izgalmas attrakciót.

Ezek csupán néhány lehetőség, de természetesen attól függően, hogy milyen az érdeklődési köröd, szívesen adok további javaslatokat is!
AI: Természetesen! A Práter Bécs híres vidámparkja, amely a város egyik legnépszerűbb turisztikai látványossága. A Práter területe egy hatalmas zöld park, am

**Magyarázat:**
* Az LCEL alapú megközelítés (`RunnablePassthrough`, `|` operátor) rugalmasabb, mint a régi `ConversationChain`.
* A `ConversationBufferMemory` egyszerű, de nagy beszélgetéseknél memórialimitekbe ütközhet. Ilyenkor más típusok (pl. `ConversationBufferWindowMemory` - csak az utolsó K üzenetet tárolja, `ConversationSummaryMemory` - összefoglalja a beszélgetést) hasznosabbak lehetnek.
* A `memory_key` (`"chat_history"`) meg kell egyezzen a `MessagesPlaceholder` `variable_name`-ével.
* A `save_context` biztosítja, hogy a beszélgetés mindkét oldala (felhasználó és AI) el legyen tárolva a következő fordulóhoz.

---

## 6. Retrieval-Augmented Generation (RAG)

A RAG lehetővé teszi az LLM-ek számára, hogy külső tudásbázisból (pl. dokumentumokból) származó információkat használjanak fel a válaszadás során, kiegészítve ezzel a saját, tréning során tanult tudásukat. Ez különösen hasznos specifikus, naprakész vagy privát adatokkal való munkához.

**Folyamat:**
1.  **Dokumentumok betöltése:** Egy `DocumentLoader` (pl. `PyPDFLoader`) beolvassa a forrásfájlt (pl. PDF).
2.  **Szöveg darabolása (Chunking):** Egy `TextSplitter` (pl. `RecursiveCharacterTextSplitter`) kisebb, kezelhető méretű darabokra (chunkokra) vágja a szöveget, gyakran kis átfedéssel.
3.  **Beágyazások (Embeddings) készítése:** Egy Embedding modell (pl. `OpenAIEmbeddings`) a szövegdarabokat numerikus vektorokká alakítja, amelyek a szöveg szemantikai jelentését kódolják.
4.  **Vektoradatbázis létrehozása:** Egy vektortár (pl. `FAISS`, `Chroma`) tárolja a vektorokat és lehetővé teszi a gyors hasonlósági keresést.
5.  **Lekérdező (Retriever) létrehozása:** A vektortár alapján létrehozunk egy retrievert, ami egy kérdéshez képes megtalálni a legrelevánsabb dokumentumdarabokat.
6.  **RAG Lánc Építése:** Az LCEL segítségével összekapcsoljuk a retrievert, a memóriát és az LLM-et egy lánccá, ami képes a kérdés alapján releváns információkat lekérni és azok felhasználásával válaszolni.

**Példa (Javított LCEL alapú RAG lánc memóriával):**

In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, get_buffer_string
from langchain_core.output_parsers import StrOutputParser # <-- Fontos import
from operator import itemgetter # <-- Szótár elemek eléréséhez
from google.colab import userdata
import warnings # Az esetleges letöltési figyelmeztetések kezeléséhez

# Figyelmeztetések szűrése (pl. FAISS letöltéskor)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Setup (LLM, Memory, Loader, Splitter, Embeddings) ---
# Győződj meg róla, hogy az OPENAI_API_KEY be van állítva
llm_rag = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model="gpt-3.5-turbo")

# Új memória objektum a RAG lánchoz
memory_rag = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key='answer')

# Dokumentum betöltése (Cseréld le egy létező PDF URL-re vagy helyi elérési útra)
# Példa
pdf_url = 'https://www.wien.info/resource/blob/412434/49dcd5d77c516144b39195951e771b83/route-wege-die-stadt-kennen-zu-lernen-en-data.pdf#'
try:
    print(f"PDF letöltése innen: {pdf_url}")
    loader = PyPDFLoader(pdf_url)
    raw_documents = loader.load()
    print(f"Dokumentum betöltve, {len(raw_documents)} oldal.")
except Exception as e:
    print(f"Hiba a PDF betöltése közben: {e}")
    print("Kérlek, adj meg egy érvényes PDF URL-t vagy fájl elérési utat a 'pdf_url' változóban.")
    # Itt leállíthatod a futást vagy használhatsz dummy adatokat
    raw_documents = [] # Üres lista, hogy a kód tovább fusson, de a RAG nem fog működni

if raw_documents:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    documents = text_splitter.split_documents(raw_documents)
    print(f"Dokumentum felosztva {len(documents)} darabra (chunk).")

    # Győződj meg róla, hogy az OPENAI_API_KEY be van állítva
    embeddings = OpenAIEmbeddings(api_key=userdata.get('OPENAI_API_KEY'))

    # Vektoradatbázis létrehozása (időbe telhet)
    print("Vektoradatbázis létrehozása...")
    vectorstore = FAISS.from_documents(documents, embeddings)
    print("Vektoradatbázis kész.")

    # Retriever létrehozása
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3}) # 'k': hány releváns chunkot kérjünk le
    print("Retriever kész.")
else:
    print("Nincsenek dokumentumok a RAG lánc építéséhez.")
    retriever = None # Hogy a kód ne álljon le, de a RAG nem működik

# --- Prompts ---
# Prompt az önálló kérdés generálásához (memória felhasználásával)
condense_question_prompt = ChatPromptTemplate.from_messages([
    ("system", "Adott a következő beszélgetés és egy követő kérdés, fogalmazza át a követő kérdést önálló kérdéssé, a keresési lekérdezéshez szükséges alapvető információkra összpontosítva.\n\nChat History:\n{chat_history}\nFollow Up Input: {input}\nStandalone question:"),
])

# Prompt a végső válasz generálásához (kontextus + önálló kérdés alapján)
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Ön egy segítőkész asszisztens. Válaszoljon a kérdésre kizárólag a következő megadott szövegkörnyezet alapján.\nHa a szövegkörnyezet nem tartalmazza a választ, mondja azt, hogy a megadott dokumentumokban nincs erről információja.\n\nContext:\n{context}\n\nQuestion: {standalone_question}\n\nAnswer:"),
])

# --- RAG Lánc LCEL-lel ---

# Függvény a dokumentumok formázásához (stringgé alakítás)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Függvény a memóriából a string előzmény kinyeréséhez
def get_chat_history(inputs):
    res = memory_rag.load_memory_variables(inputs)
    return get_buffer_string(res.get('chat_history', []))

# Lánc definiálása csak akkor, ha a retriever sikeresen létrejött
if retriever:
    # 1. Lépés: Először add hozzá a chat_history-t az inputhoz
    step1_add_history = RunnablePassthrough.assign(
        chat_history=RunnableLambda(get_chat_history)
    )

    # 2. Lépés: Most generáld a standalone kérdést, felhasználva az inputot ÉS a history-t
    step1_generate_standalone = step1_add_history.assign(
        standalone_question=(
            condense_question_prompt
            | llm_rag
            | StrOutputParser()
        )
    )

    # 3. Lépés: Keresd meg a dokumentumokat a standalone kérdés alapján
    step2_retrieve_docs = step1_generate_standalone.assign(
        # Az itemgetter a bemeneti szótárból veszi a standalone_question-t
        # A retriever megkeresi a releváns dokumentumokat
        # A format_docs stringgé alakítja a dokumentumokat
        context=itemgetter("standalone_question") | retriever | format_docs
    )

    # 4. Lépés: Generálj választ a kontextus és a standalone kérdés alapján
    rag_chain = step2_retrieve_docs.assign(
        answer=answer_prompt | llm_rag | StrOutputParser()
    )
    print("RAG lánc sikeresen létrehozva.")
else:
    rag_chain = None
    print("RAG lánc nem jött létre, mert a retriever inicializálása sikertelen volt.")


# --- Beszélgetés RAG-gal és memóriával ---
def chat_rag_with_memory(user_input):
    if not rag_chain:
        return "Sajnos a RAG lánc nem érhető el a dokumentumok betöltési hibája miatt."

    # A lánc a teljes szótárat adja vissza
    result_dict = rag_chain.invoke({"input": user_input})

    # Memória mentése (eredeti input és a generált válasz string)
    memory_rag.save_context({"input": user_input}, {"answer": result_dict['answer']})

    # Csak a válasz stringet adjuk vissza
    return result_dict['answer']

# --- Használat Példa ---
# Csak akkor futtasd, ha a RAG lánc létrejött
if rag_chain:
    print("\n--- RAG Teszt ---")
    # Az első kérdésnél még nincs memória
    print("Felhasználó: Mi a Mozart ház érdekessége?")
    print("AI:", chat_rag_with_memory("Mi a Mozart ház érdekessége?"))

    # A második kérdésnél már használja a memóriát az önálló kérdés generálásához
    # print("\nFelhasználó: És mi található a közelében?")
    # print("AI:", chat_rag_with_memory("És mi található a közelében?"))
else:
    print("\nA RAG lánc nem tesztelhető a korábbi hiba miatt.")

PDF letöltése innen: https://www.wien.info/resource/blob/412434/49dcd5d77c516144b39195951e771b83/route-wege-die-stadt-kennen-zu-lernen-en-data.pdf#
Dokumentum betöltve, 20 oldal.
Dokumentum felosztva 70 darabra (chunk).
Vektoradatbázis létrehozása...
Vektoradatbázis kész.
Retriever kész.
RAG lánc sikeresen létrehozva.

--- RAG Teszt ---
Felhasználó: Mi a Mozart ház érdekessége?
AI: A Mozart-házat az teszi érdekessé, hogy az eredeti, díszes lakásán tartotta a nagy zeneszerző partijait 1784 és 1787 között.


**Magyarázat:**
* Ez a komplex lánc először átalakítja a felhasználó kérdését (a beszélgetési előzményt is figyelembe véve) egy önálló kérdéssé.
* Ezután ezzel az önálló kérdéssel keres a vektoradatbázisban releváns információk után (`retriever`).
* Végül a talált információkat (kontextus) és az önálló kérdést átadja az LLM-nek, hogy csak ezen információk alapján generáljon választ.
* A `StrOutputParser` biztosítja, hogy az LLM kimenetei stringek legyenek a lánc további lépéseihez.
* A `RunnablePassthrough.assign` és az `itemgetter` segít a szükséges adatok (input, chat\_history, standalone\_question, context) továbbításában a lánc lépései között.
* Hibakezelést adtunk hozzá a PDF betöltéséhez és a retriever létrehozásához.

---

## 7. Retriever eszközként ügynökben

Lehetőség van arra is, hogy a retrievert (ami a specifikus tudásbázisunkból keres) egy eszközként (`Tool`) adjuk hozzá egy ügynökhöz. Így az ügynök eldöntheti, hogy a kérdés megválaszolásához a belső tudását, egy általános kereső eszközt (pl. Google Search), vagy a specifikus tudásbázisunkat (retriever) használja-e.

**Példa (LCEL alapú ügynök retriever eszközzel):**

In [13]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain.tools.retriever import create_retriever_tool # <-- Retriever eszköz létrehozásához
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.memory import ConversationBufferMemory # Memória az ügynökhöz
from google.colab import userdata

# --- Setup (LLM, retriever - mint korábban) ---
# Győződj meg róla, hogy az OPENAI_API_KEY be van állítva
llm_agent_rag = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model="gpt-3.5-turbo", temperature=0) # temperature=0 a következetesebb válaszokért

# Feltételezzük, hogy a 'retriever' változó létezik az előző RAG lépésből
# Ha nem, akkor a retriever_tool nem fog működni!
if not retriever:
    print("Hiba: A 'retriever' objektum nem létezik. A retriever eszköz nem hozható létre.")
    retriever_tool = None
else:
    # 1. Retriever Tool létrehozása
    retriever_tool = create_retriever_tool(
        retriever,
        "austria_guide_search", # Eszköz neve - legyen beszédes!
        "Akkor használd, ha Ausztriával, Béccsel, vagy az osztrák turisztikai PDF dokumentum tartalmával kapcsolatos specifikus kérdésekre keresel választ. Például: Mozart ház, bécsi látnivalók a dokumentumból." # Nagyon fontos a jó leírás!
    )

# 2. Általános kereső eszköz
# Győződj meg róla, hogy a SERPAPI_API_KEY be van állítva
search_agent = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))
search_tool = Tool(
    name="General_Search", # Különböző név kell!
    func=search_agent.run,
    description="Akkor használd, ha aktuális információkra, időjárásra, vagy olyan általános kérdésekre van szükséged, amelyek valószínűleg nincsenek az osztrák turisztikai dokumentumban."
)

# 3. Eszközök listája
# Csak a sikeresen létrehozott eszközöket adjuk hozzá
tools_agent = [search_tool]
if retriever_tool:
    tools_agent.append(retriever_tool)

# 4. Ügynök létrehozása (ReAct példa)
# Használjunk beszélgetésre optimalizált ReAct promptot
try:
    prompt_agent_rag = hub.pull("hwchase17/react-chat")
except Exception as e:
    print(f"Hiba a prompt letöltésekor a Langchain Hub-ról: {e}")
    prompt_agent_rag = None # Hogy a kód ne álljon le

if prompt_agent_rag and tools_agent: # Csak akkor hozzuk létre az ügynököt, ha a prompt és eszközök is rendben vannak
    agent_rag_tool = create_react_agent(llm_agent_rag, tools_agent, prompt_agent_rag)

    # 5. Memória az ügynökhöz
    # Az AgentExecutor kezeli a memória mentését/betöltését, ha a prompt támogatja.
    # A "hwchase17/react-chat" prompt várja a 'chat_history' változót.
    memory_agent_rag = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # 6. Agent Executor létrehozása memóriával
    agent_executor_rag_tool = AgentExecutor(
        agent=agent_rag_tool,
        tools=tools_agent,
        memory=memory_agent_rag, # Executor kezeli a memóriát
        verbose=True,
        handle_parsing_errors=True # Ajánlott
    )
    print("Ügynök RAG eszközzel sikeresen létrehozva.")
else:
    agent_executor_rag_tool = None
    print("Az ügynök létrehozása sikertelen volt a hiányzó prompt vagy eszközök miatt.")


# --- Használat Példa ---
if agent_executor_rag_tool:
    print("\n--- Ügynök Teszt (RAG eszközzel) ---")
    # Kérdés, amihez a PDF-et kellene használnia
    print("Felhasználó: Mesélj nekem a Mozartházról a bécsi útikalauz alapján!")
    response1 = agent_executor_rag_tool.invoke({"input": "Mesélj nekem a Mozartházról a bécsi útikalauz alapján!"})
    print("AI:", response1['output'])

    # Kérdés, amihez az általános keresőt kellene használnia
    print("\nFelhasználó: Milyen az időjárás most Budapesten?")
    response2 = agent_executor_rag_tool.invoke({"input": "Milyen az időjárás most Budapesten?"})
    print("AI:", response2['output'])

    # Kérdés, ami a memóriára is támaszkodhat
    print("\nFelhasználó: Említetted a Mozartházat. Mikor rendezett ott partikat?")
    response3 = agent_executor_rag_tool.invoke({"input": "Említetted a Mozartházat. Mikor rendezett ott partikat?"})
    print("AI:", response3['output'])
else:
    print("\nAz ügynök nem tesztelhető a korábbi hiba miatt.")

Ügynök RAG eszközzel sikeresen létrehozva.

--- Ügynök Teszt (RAG eszközzel) ---
Felhasználó: Mesélj nekem a Mozartházról a bécsi útikalauz alapján!


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: austria_guide_search
Action Input: MozartházWestart with the MozarthausViennaexperience.
Its centerpiece, Mozart’sopulent originalapart-
ment on the grand Beletage or piano nobile,
hosted many of the great composer’s parties
between 1784 and 1787. And now to a modern-
era venue: anyone in search of jazz in Vienna
will stumble upon Porgy & Bess sooner or later, where there’s live
music to enjoy virtually every day.Once a legendary erotic cinema,
today Porgy has a reputation as the most beautiful jazz club in
Europe. Meanwhile, at the Ronacher, a beacon of Vienna‘s theater
scene, it‘s showtime for big musical productions. Up next is Vienna’s
sound museum, the House of Music. Dull museums are a thing of
the past – feel free to join in here: wield the conduct

**Magyarázat:**
* A `create_retriever_tool` egy speciális függvény, ami a meglévő retrieverünkből készít egy Langchain `Tool`-t.
* Kulcsfontosságú a két eszköz (`austria_guide_search` és `General_Search`) `description`-jének megkülönböztethetősége. Az ügynök ez alapján dönti el, mikor melyiket vesse be. Minél pontosabb a leírás, annál jobban fog működni az ügynök.
* A `hwchase17/react-chat` prompt kifejezetten beszélgetésekhez készült, és tartalmazza a `chat_history` kezelését, így az `AgentExecutor` `memory` paraméterével használható.
* Az ügynök így képes komplexebb, több forrásból származó információt igénylő kérdéseket is megválaszolni, dinamikusan választva a megfelelő eszközt.


```

Beadandó feladat:

Langchain segítségével egy ügynök létrehozása. Az ügynöknek 3 eszközt kell használnia,

RAG egy vektoradatbázisból,

egy internetes kereső

egy nagy nyelvi modell.

A vektoradatbázis legalább 10 dokumentumot kell tartalmazzon. Be kell mutatni az ügynök működését mind a három eszköz használatával kapcsolatosan. Aki 5-ös érdemjegyet szeretne annak építeni kell az ügynökhöz egy webes felületet tetszőleges megoldással (flask, Gradio, …).